In [1]:
! pip install PyMySQL

     |████████████████████████████████| 43 kB 1.5 MB/s 


In [2]:
import pymysql.cursors

In [3]:
# 데이터베이스 연결
connection = pymysql.connect(host = '13.124.77.49',
                             user='USER_SQLSTK',
                             password='1qaz2wsx',
                             database='DB_SQLSTK',
                             charset='utf8',
                             cursorclass=pymysql.cursors.DictCursor                             
                             )

OperationalError: ignored

In [ ]:
connection

In [ ]:
cursor = connection.cursor()

In [ ]:
cursor

In [ ]:
# 실행할 SQL
# 2019년 1월 1일 ~ 2020년 12월 31일 삼성전자 종가, 시가, 고가, 저가, 거래량, (5, 10, 20) 이평선
sql = """
      SELECT T1.STK_CD, T1.STK_NM, T1.SEC_NM
            ,T2.DT, O_PRC, T2.L_PRC, T2.H_PRC, T2.C_PRC, T2.VOL
            ,M5_PRC, M10_PRC, M20_PRC
      FROM stock T1
      INNER JOIN HISTORY_DT T2
      ON (T2.STK_CD = T1.STK_CD)
      WHERE T1.STK_NM = '삼성전자'
      AND T2.DT >= '2019-08-01'
      AND T2.DT <= '2020-03-31';
  """

In [ ]:
# SQL 쿼리를 실행할 준비
cursor.execute(sql)

In [ ]:
# SQL 쿼리를 실행하고 결과 result
result = cursor.fetchall()

In [ ]:
# sql 쿼리 실행 결과 출력
result

In [ ]:
# sql을 데이터프레임으로 
import pandas as pd
df = pd.DataFrame(result)
df.head()

In [ ]:
import pandas as pd
# from datetime import datetime
df1 = pd.read_csv("samsung_m60_prc.csv", engine="python") # 삼성전자 60일 이평선이 들어간 csv
df1['60days'] = df1['c_prc']/df1['M60_PRC'] * 100 # 60일 이평선의 이격도
df['M60_PRC'] = df1['M60_PRC'] # 기존 df에 m60_prc 라는 열을 만들고 그 안에 df1에서 가져온 60일 이평선 대입
df['20days'] = df['C_PRC']/df['M20_PRC']*100 # 20일 이평선의 이격도
df['60days'] = df1['60days'] # 기존 df에 60일 이격도 대입
df1.head()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# 그래프에 넣을 때 오류를 방지하기 위해 date 타입으로 변경
df["DT"]=pd.to_datetime(df["DT"])
# O_PRC 정수로 변경
df.loc[: , "O_PRC":] = df.loc[: , "O_PRC":].astype(int)

In [ ]:
df.info()

In [ ]:
! pip install plotly==5.2.1

In [ ]:
import plotly.graph_objects as go

In [ ]:
# 봉차트로 시각화
candle = go.Candlestick(
        x=df.DT,              #날짜
        open=df['O_PRC'],      # 시가
        high=df['H_PRC'],     #고가
        low=df['L_PRC'],       #저가
        close=df['C_PRC'],   # 종가
        increasing_line_color='red', # 상승봉의 색 설정
	      decreasing_line_color='blue' # 하락봉의 색 설정
)

ch = go.Figure(data=candle) # 봉차트 candle을 그릴 준비를 함
ch.show() # 봉차트 출력

In [ ]:
# 봉차트로 시각화
candle = go.Candlestick(
        x=df.DT,              #날짜
        open=df['O_PRC'],      # 시가
        high=df['H_PRC'],     #고가
        low=df['L_PRC'],       #저가
        close=df['C_PRC'],   # 종가
        increasing_line_color='red', # 상승봉의 색 설정
	      decreasing_line_color='blue' # 하락봉의 색 설정
)

# 바 차트(거래량) 객체 생성
volume_bar = go.Bar(x=df.DT, y=df['VOL'])

In [ ]:
import plotly.subplots as ms

In [ ]:
# 2줄 1칸 차트를 생성
# x축은 공유함
fig = ms.make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)

# row=1, col=1 : 1번째줄 1번째 칸에 candle 차트 출력
fig.add_trace(candle, row=1, col=1)
# row=2, col=1 : 2번째줄 1번째 칸에 volume_bar 차트 출력
fig.add_trace(volume_bar, row=2, col=1)

# row=1, col=1 : 1번째줄 1번째 칸에 "Stock Price" 제목 출력
fig.update_yaxes(title_text="Stock Price", row=1, col=1)
# row=2, col=1 : 2번째줄 1번째 칸에 "Volume" 제목 출력
fig.update_yaxes(title_text="Volume", row=2, col=1)
# Samsung stock price 제목 출력
fig.update_layout( title='삼성전자' )

In [4]:
# 봉차트로 시각화
candle = go.Candlestick(
        x=df.DT,              #날짜
        open=df['O_PRC'],      # 시가
        high=df['H_PRC'],     #고가
        low=df['L_PRC'],       #저가
        close=df['C_PRC'],   # 종가
        increasing_line_color='red', # 상승봉의 색 설정
	      decreasing_line_color='blue' # 하락봉의 색 설정
)

# 바 차트(거래량) 객체 생성
volume_bar = go.Bar(x=df.DT, y=df['VOL'])

# 5일 이동 평균 선 차트 
ma5 = go.Scatter(x=df.DT, y=df['M5_PRC'], line=dict(color='black', width=0.8), name='ma5')
# 10일 이동 평균 선 차트 
ma10 = go.Scatter(x=df.DT, y=df['M10_PRC'], line=dict(color='red', width=0.9), name='ma10')
# 20일 이동 평균 선 차트 
ma20 = go.Scatter(x=df.DT, y=df['M20_PRC'], line=dict(color='green', width=1), name='ma20')
# 60일 이동 평균 선 차트
ma60 = go.Scatter(x=df.DT, y=df['M60_PRC'], line=dict(color='purple', width=2), name='ma60')
# 20일 이격도 차트
day20 = go.Scatter(x=df.DT, y=df['20days'], line=dict(color='green', width=2), name='day20')
# 60일 이격도 차트
day60 = go.Scatter(x=df.DT, y=df['60days'], line=dict(color='purple', width=2), name='day60')

NameError: ignored

In [ ]:
# 2줄 1칸 차트를 생성
# x축은 공유함
fig = ms.make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02)

# row=1, col=1 : 1번째줄 1번째 칸에 candle 차트 출력
fig.add_trace(candle, row=1, col=1)
# row=1, col=1 : 1번째줄 1번째 칸에 ma5 차트 출력
fig.add_trace(ma5, row=1, col=1)
# row=1, col=1 : 1번째줄 1번째 칸에 ma10 차트 출력
fig.add_trace(ma10, row=1, col=1)
# row=1, col=1 : 1번째줄 1번째 칸에 ma20 차트 출력
fig.add_trace(ma20, row=1, col=1)
# orw=1, col=1 : 1번째 줄 1번째 칸에 ma60 차트 출력
fig.add_trace(ma60, row=1, col=1)

# row=2, col=1 : 2번째줄 1번째 칸에 volume_bar 차트 출력
fig.add_trace(volume_bar, row=2, col=1)
# row=3, col=1 : 3번째줄 1번째 칸에 20days 차트 출력
fig.add_trace(day20, row=3, col=1)
# row=3, col=1 : 3번째줄 1번째 칸에 60days 차트 출력
fig.add_trace(day60, row=3, col=1)
# 이평선 90 이하일 때 매수
fig.add_hline(y=90, line=dict(color='blue', width=2), name='cutline90') 
# 이평선 110 이상일 때 매수
fig.add_hline(y=110, line=dict(color='red', width=2), name='cutline110') 

# row=1, col=1 : 1번째줄 1번째 칸에 "주가" 제목 출력
fig.update_yaxes(title_text="주가", row=1, col=1)
# row=2, col=1 : 2번째줄 1번째 칸에 "거래량" 제목 출력
fig.update_yaxes(title_text="거래량", row=2, col=1)
# row=3, col=1 : 3번째줄 1번째 칸에 "이격도" 제목 출력
fig.update_yaxes(title_text="이격도", row=3, col=1)

# 삼성전자 제목 출력
fig.update_layout( title='삼성전자')

#최종

In [ ]:
# 이격도 차트
fig = ms.make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.02)
# row=3, col=1 : 3번째줄 1번째 칸에 20days 차트 출력
fig.add_trace(day20, row=1, col=1)
# row=3, col=1 : 3번째줄 1번째 칸에 60days 차트 출력
fig.add_trace(day60, row=1, col=1)
# 이평선 90 이하일 때 매수
fig.add_hline(y=90, line=dict(color='blue', width=2), name='cutline90') 
# 이평선 110 이상일 때 매수
fig.add_hline(y=110, line=dict(color='red', width=2), name='cutline110') 
# 삼성전자 제목 출력
# fig.update_layout( title='삼성전자')

In [ ]:
# 봉차트 + 거래량
# 2줄 1칸 차트를 생성
# x축은 공유함
fig = ms.make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)

# row=1, col=1 : 1번째줄 1번째 칸에 candle 차트 출력
fig.add_trace(candle, row=1, col=1)
# row=2, col=1 : 2번째줄 1번째 칸에 volume_bar 차트 출력
fig.add_trace(volume_bar, row=2, col=1)

# row=1, col=1 : 1번째줄 1번째 칸에 "Stock Price" 제목 출력
fig.update_yaxes(title_text="Stock Price", row=1, col=1)
# row=2, col=1 : 2번째줄 1번째 칸에 "Volume" 제목 출력
fig.update_yaxes(title_text="Volume", row=2, col=1)
# Samsung stock price 제목 출력
fig.update_layout( title='삼성전자' )